In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_test = pd.read_csv("../input/tweet-sentiment-extraction/test.csv")
df_test.head(2)

In [ ]:
sub_df = pd.read_csv('../input/tweet-sentiment-extraction/sample_submission.csv')
sub_df.head(2)

In [ ]:
sub_df.shape

In [ ]:
from transformers import AutoTokenizer

# using roberta-base tokenizer to tokenize the text into input IDs that model can make sense of.

tokenizer = AutoTokenizer.from_pretrained("../input/robertatokenizer/",local_files_only =True)

In [ ]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained("../input/robertamodel",local_files_only=True)

In [ ]:
from datasets import Dataset

df_test.reset_index(drop=True,inplace=True)

test_data = Dataset.from_pandas(df_test)
test_data

In [ ]:
MAX_LENGTH = 73

def post_porocess_data(examples):
  questions = examples["sentiment"]
  context = examples["text"]
  inputs = tokenizer(
      questions,
      context,
      max_length = MAX_LENGTH,
      padding="max_length",
      return_offsets_mapping = True,   
  )

  for i in range(len(inputs["input_ids"])):
    offset = inputs["offset_mapping"][i]
    sequence_ids = inputs.sequence_ids(i)
    inputs["offset_mapping"][i] = [
                                  o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
    ]
  return inputs

In [ ]:
processed_test_data = test_data.map(
    post_porocess_data,
    batched = True
)
processed_test_data

In [ ]:
tf_test_dataset = processed_test_data.to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    shuffle=False,
    batch_size=16,
)

In [ ]:
outputs = model.predict(tf_test_dataset)

start_logits = outputs.start_logits
end_logits = outputs.end_logits

In [ ]:
n_best = 20

def predict_answers(inputs):
    predicted_answer = []
    for i in range(len(inputs["offset_mapping"])):
        start_logit = inputs["start_logits"][i]
        end_logit = inputs["end_logits"][i]
        context = inputs["text"][i]
        offset = inputs["offset_mapping"][i]
        start_indexes = np.argsort(start_logit)[-1: -n_best - 1:-1].tolist()
        end_indexes = np.argsort(end_logit)[-1: -n_best - 1: -1].tolist()
        
        flag = False
        for start_index in start_indexes:
            for end_index in end_indexes:
                # skip answer that are not in the context.
                if offset[start_index] is None or offset[end_index] is None:
                    continue
                # skip answer with length that is either < 0
                if end_index < start_index:
                    continue
                flag = True
                answer = context[offset[start_index][0]: offset[end_index][1]]
                predicted_answer.append(answer)
                break
            if flag:
                break
        if not flag:
            predicted_answer.append(answer)
    return {"predicted_answer":predicted_answer}


In [ ]:
processed_test_data.set_format("pandas")

In [ ]:
processed_test_df =  processed_test_data[:]
processed_test_df.head(2)

In [ ]:
processed_test_df["start_logits"] = start_logits.tolist()
processed_test_df["end_logits"] = end_logits.tolist()
processed_test_df.shape

In [ ]:
processed_test_df["text"] = df_test.text
processed_test_df.head(2)

In [ ]:
final_test_data = Dataset.from_pandas(processed_test_df)
final_test_data

In [ ]:
final_test_data = final_test_data.map(predict_answers,batched=True)
final_test_data

In [ ]:
#predictions_df = pd.DataFrame.from_dict(predictions)
sub_df['selected_text'] = final_test_data["predicted_answer"]

sub_df.to_csv('submission.csv', index=False)

print("File submitted successfully.")